<a href="https://colab.research.google.com/github/HamzaNasiem/AI_Agent_essay_grading_system/blob/main/essay_grading_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Essay Grading System using LangGraph

In [40]:
from google.colab import userdata
import os

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')


In [41]:
from typing import TypedDict, List
from langgraph.graph import StateGraph, END
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
import os
from dotenv import load_dotenv
import re
import logging

# Load environment variables and set OpenAI API key
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

# Set up logging
logging.basicConfig(level=logging.INFO)


# State Definition

This cell defines the State class, which represents the state of our grading process.

In [42]:
class State(TypedDict):
    """Represents the state of the essay grading process."""
    essay: str
    relevance_score: float
    grammar_score: float
    structure_score: float
    depth_score: float
    final_score: float

# Language Model Initialization
This cell initializes the ChatOpenAI model.

In [43]:
# Initialize the ChatOpenAI model
llm = ChatOpenAI(model="gpt-4o-mini")

# Grading Functions
This cell defines the functions used in the grading process, including score extraction, individual grading components, and feedback generation.



In [44]:
def extract_score(content: str) -> float:
    """Extract the numeric score from the LLM's response."""
    match = re.search(r'Score:\s*(\d+(\.\d+)?)', content)
    if match:
        return float(match.group(1))
    raise ValueError(f"Could not extract score from: {content}")

def check_relevance(state: State) -> State:
    """Check the relevance of the essay."""
    prompt = ChatPromptTemplate.from_template(
        "Analyze the relevance of the following essay to the given topic. "
        "Provide a relevance score between 0 and 1. "
        "Your response should start with 'Score: ' followed by the numeric score, "
        "then provide your explanation.\n\nEssay: {essay}"
    )
    result = llm.invoke(prompt.format(essay=state["essay"]))
    try:
        state["relevance_score"] = extract_score(result.content)
    except ValueError as e:
        print(f"Error in check_relevance: {e}")
        state["relevance_score"] = 0.0
    return state

def check_grammar(state: State) -> State:
    """Check the grammar of the essay."""
    prompt = ChatPromptTemplate.from_template(
        "Analyze the grammar and language usage in the following essay. "
        "Provide a grammar score between 0 and 1. "
        "Your response should start with 'Score: ' followed by the numeric score, "
        "then provide your explanation.\n\nEssay: {essay}"
    )
    result = llm.invoke(prompt.format(essay=state["essay"]))
    try:
        state["grammar_score"] = extract_score(result.content)
    except ValueError as e:
        print(f"Error in check_grammar: {e}")
        state["grammar_score"] = 0.0
    return state

def analyze_structure(state: State) -> State:
    """Analyze the structure of the essay."""
    prompt = ChatPromptTemplate.from_template(
        "Analyze the structure of the following essay. "
        "Provide a structure score between 0 and 1. "
        "Your response should start with 'Score: ' followed by the numeric score, "
        "then provide your explanation.\n\nEssay: {essay}"
    )
    result = llm.invoke(prompt.format(essay=state["essay"]))
    try:
        state["structure_score"] = extract_score(result.content)
    except ValueError as e:
        print(f"Error in analyze_structure: {e}")
        state["structure_score"] = 0.0
    return state

def evaluate_depth(state: State) -> State:
    """Evaluate the depth of analysis in the essay."""
    prompt = ChatPromptTemplate.from_template(
        "Evaluate the depth of analysis in the following essay. "
        "Provide a depth score between 0 and 1. "
        "Your response should start with 'Score: ' followed by the numeric score, "
        "then provide your explanation.\n\nEssay: {essay}"
    )
    result = llm.invoke(prompt.format(essay=state["essay"]))
    try:
        state["depth_score"] = extract_score(result.content)
    except ValueError as e:
        print(f"Error in evaluate_depth: {e}")
        state["depth_score"] = 0.0
    return state

def calculate_final_score(state: State) -> State:
    """Calculate the final score based on individual component scores."""
    state["final_score"] = (
        state["relevance_score"] * 0.3 +
        state["grammar_score"] * 0.2 +
        state["structure_score"] * 0.2 +
        state["depth_score"] * 0.3
    )
    return state

# Workflow Definition
This cell defines the grading workflow using StateGraph.

In [45]:
# Initialize the StateGraph
workflow = StateGraph(State)

# Add nodes to the graph
workflow.add_node("check_relevance", check_relevance)
workflow.add_node("check_grammar", check_grammar)
workflow.add_node("analyze_structure", analyze_structure)
workflow.add_node("evaluate_depth", evaluate_depth)
workflow.add_node("calculate_final_score", calculate_final_score)

# Define and add conditional edges
workflow.add_conditional_edges(
    "check_relevance",
    lambda x: "check_grammar" if x["relevance_score"] > 0.5 else "calculate_final_score"
)
workflow.add_conditional_edges(
    "check_grammar",
    lambda x: "analyze_structure" if x["grammar_score"] > 0.6 else "calculate_final_score"
)
workflow.add_conditional_edges(
    "analyze_structure",
    lambda x: "evaluate_depth" if x["structure_score"] > 0.7 else "calculate_final_score"
)
workflow.add_conditional_edges(
    "evaluate_depth",
    lambda x: "calculate_final_score"
)

# Set the entry point
workflow.set_entry_point("check_relevance")

# Set the exit point
workflow.add_edge("calculate_final_score", END)

# Compile the graph
app = workflow.compile()

# Essay Grading Function
This cell defines the main function to grade an essay using the defined workflow.

In [46]:
def grade_essay(essay: str) -> dict:
    """Grade the given essay using the defined workflow."""
    initial_state = State(
        essay=essay,
        relevance_score=0.0,
        grammar_score=0.0,
        structure_score=0.0,
        depth_score=0.0,
        final_score=0.0
    )
    result = app.invoke(initial_state)
    # Add feedback to the result dictionary
    result['feedback'] = "This is a placeholder for feedback." # Example feedback, replace with actual feedback generation logic
    return result

# Batch Grading Function
This cell allows for grading multiple essays at once.

In [47]:
def grade_multiple_essays(essays: List[str]) -> List[dict]:
    """Grade a list of essays and return their results."""
    results = []
    for essay in essays:
        result = grade_essay(essay)
        results.append(result)
    return results


# Example Usage
This cell demonstrates how to use the grading system with example essays.

In [49]:
# Essay lene ka function
def get_user_input() -> str:
    essay = input("Please enter your essay: ")
    return essay

# User se ek essay lein
essay_to_grade = get_user_input()

# Grade the essay
graded_result = grade_essay(essay_to_grade)

# Display the results
print("Essay Results:")
print(f"Relevance Score: {graded_result['relevance_score']}")
print(f"Grammar Score: {graded_result['grammar_score']}")
print(f"Structure Score: {graded_result['structure_score']}")
print(f"Depth Score: {graded_result['depth_score']}")
print(f"Final Score: {graded_result['final_score']}")



Please enter your essay: The Apple: A Symbol of Health and Knowledge The apple, a small yet powerful fruit, holds a significant place in cultures and cuisines around the world. Known for its crisp texture and sweet-tart flavor, apples are not just a delightful snack; they are also rich in nutrients and have been associated with numerous health benefits. From ancient myths to modern science, the apple has been a symbol of health, knowledge, and temptation.  Nutritional Value Apples are a great source of vitamins, particularly vitamin C, which is essential for immune function and skin health. They also contain dietary fiber, which aids in digestion and helps maintain a healthy weight. Eating apples regularly has been linked to a reduced risk of chronic diseases such as heart disease and diabetes. The antioxidants found in apples, including quercetin and flavonoids, help protect the body from oxidative stress and inflammation.  Cultural Significance Throughout history, apples have been in